In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [5]:
titanic = pd.read_csv('titanic.csv')
titanic.drop(['cabin','boat','body','name','home.dest','ticket'], axis=1, inplace=True)

In [6]:
# impute age based on mean of pclass age
def add_age(cols):
    age = cols[0]
    pclass = cols[1]
    if pd.isnull(age):
        return int(titanic[titanic["pclass"] == pclass]["age"].mean())
    else:
        return age
    
titanic["age"] = titanic[["age", "pclass"]].apply(add_age,axis=1)

In [ ]:
#drop all the other missing values
titanic.dropna(inplace=True)

In [ ]:
titanic.head()

In [ ]:
male = pd.get_dummies(titanic['sex'],drop_first=True)
embark = pd.get_dummies(titanic['embarked'], drop_first=True)
pclass = pd.get_dummies(titanic['pclass'], drop_first=True)

In [ ]:
titanic = pd.concat([titanic, male, embark, pclass], axis=1)

In [ ]:
X = titanic.drop(['pclass','sex','embarked','survived'], axis=1)
y = titanic['survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.35, random_state=832, stratify=y)

In [ ]:
X_train.shape

In [ ]:
y_train.mean()

In [ ]:
y_test.mean()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression(solver='liblinear')

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
y_pred_train = logreg.predict(X_train)

In [ ]:
## In sample scores
metrics.accuracy_score(y_train, y_pred_train)

In [ ]:
metrics.confusion_matrix(y_train, y_pred_train)

In [ ]:
pd.crosstab(y_train, y_pred_train)

In [ ]:
metrics.precision_score(y_train, y_pred_train)

In [ ]:
metrics.recall_score(y_train, y_pred_train)

In [ ]:
metrics.f1_score(y_train, y_pred_train)

In [ ]:
cross_val_score(logreg, X_train, y_train, cv=5).mean()

In [ ]:
cross_val_score(logreg, X_train, y_train, cv=5, scoring='precision').mean()

In [ ]:
cross_val_score(logreg, X_train, y_train, cv=5, scoring='recall').mean()

In [ ]:
cross_val_score(logreg, X_train, y_train, cv=5, scoring='f1').mean()

In [ ]:
yprobtrain = logreg.predict_proba(X_train)[:,1]

In [ ]:
pd.crosstab(y_train,yprobtrain>0.7)

In [ ]:
pd.crosstab(y_train,yprobtrain>0.5)

In [ ]:
metrics.recall_score(y_train, yprobtrain>0.3)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_train, yprobtrain)

In [ ]:
fpr[[1,5,10,20,50,100,200,250,300]]

In [ ]:
tpr[[1,5,10,20,50,100,200,250,300]]

In [ ]:
thresholds[[1,5,10,20,50,100,200,250,300]]

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc=0)
    plt.show()

In [ ]:
plot_roc_curve(fpr,tpr)

In [ ]:
def plot_pr_curve(p, r):
    plt.plot(p, r, color='orange', label='P/R Curve')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.title('Precision Recall Curve')
    plt.legend(loc=0)
    plt.show()

In [ ]:
precision, recall, thresholds = metrics.precision_recall_curve(y_train, yprobtrain)

In [ ]:
plot_pr_curve(precision, recall)

In [ ]:
np.abs(precision-recall).argmin()

In [ ]:
precision[407]

In [ ]:
recall[407]

In [ ]:
#### TEST DATA

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
metrics.confusion_matrix(y_test, y_pred)

In [ ]:
metrics.f1_score(y_test, y_pred)

In [ ]:
y_prob = logreg.predict_proba(X_test)[:,1]

In [ ]:
precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_prob)

In [ ]:
plt.plot(recall, precision)
plt.plot([0,1],[0,1])

In [ ]:
ind = np.abs(precision-recall).argmin()

In [ ]:
thresholds[ind]

In [ ]:
precision[ind]

In [ ]:
recall[ind]

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)

In [ ]:
plot_roc_curve(fpr,tpr)

In [ ]:
metrics.roc_auc_score(y_test, y_prob)